# Visualizing NYC Taxi Trips with Datashader, Altair, and Panel

In this notebook, we create an example dashboard visualizing NYC taxi trip data. The dashboard includes:

- a heatmap of the taxi trips using Holoviews and Datashader
- a histogram of the passenger counts using Altair
- lots of Panel-based widgets for controlling the map aesthetics
- a Holoviews selection stream that links the map x/y ranges to the histogram plot

As the user zooms in/out of the map, the Altair chart updates to show only data currently within the map viewport.

This dashboard is adapted from the [example dashboard](https://examples.pyviz.org/datashader_dashboard/dashboard.html) on the Datashader documentation.


#### References

For more information, see the documentation:

- [Documentation homepage](https://panel.holoviz.org)
- [App Gallery](https://panel.holoviz.org/gallery/index.html)
    - Examples of end-to-end web apps using Panel
- [Component Gallery](https://panel.holoviz.org/reference/index.html)
    - Examples (code snippets) for the types of panes, widgets, and layouts possible in Panel dashboards
- [Datashader dashboard documentation](https://examples.pyviz.org/datashader_dashboard/dashboard.html)
    - Including a detailed walk-through of many of the steps used in this dashboard

In [1]:
import panel as pn

pn.extension("vega", sizing_mode="stretch_width")  # Enables support for vega/altair

In [2]:
import numpy as np
import pandas as pd
import altair as alt

import param as pm
from colorcet import cm

import holoviews as hv
import geoviews as gv
from holoviews.operation.datashader import rasterize, shade

## Load the full data set

We'll use Pandas to load a data set of over 1 million taxi trips in NYC. The data is available for download
at:

> https://s3.amazonaws.com/datashader-data/nyc_taxi_wide.parq 


**Important** 

If you are working locally, you must download the data and place it in the `data/` folder, but if you are working on Binder, the data should already be available.

In [3]:
# select only a subset of columns
usecols = ['dropoff_x','dropoff_y','pickup_x','pickup_y','dropoff_hour','passenger_count']

# Read the data 
DATA = pd.read_parquet('data/nyc_taxi_wide.parq', engine='fastparquet', columns=usecols)

# Trim to 500,000 to avoid memory issues on Binder
N = 500000
DATA = DATA.sample(N)

**Note**

We'll load the *full* data above, and when the user changes input parameters, we'll filter the full data set in our app according to those parameters.

## Build the app

We'll define our app as a custom Python class that defines the various components of our dashboard, which include:

- The parameters we want the user to be able to change.
- Functions to generate the various charts/maps in our dashboard, based on those input parameters.
- The dependencies between our chart functions and parameters.

In [4]:
DATA.head()

,dropoff_x,dropoff_y,pickup_x,pickup_y,dropoff_hour,passenger_count
9984412,-8225882.0,4974035.5,-8228474.0,4974483.0,15,1
482085,-8230604.0,4976471.5,-8227749.0,4976001.5,7,5
4407085,-8235432.5,4976714.5,-8235519.0,4975219.0,8,1
9703980,-8236913.0,4975843.5,-8237392.5,4971979.0,18,1
3169663,-8224618.5,4976725.5,-8237412.0,4973383.0,22,1


In [5]:
# The colormaps we can choose from
cmaps = ['fire','bgy','bgyw','bmy','gray','kbc']

# Define the options for the basemap tiles that we'll use
opts = dict(xaxis=None, yaxis=None, bgcolor="black", show_grid=False)

In [6]:
class NYCTaxiApp(pm.Parameterized):
    """
    A Panel-based dashboard app visualizing data for over a million
    NYC taxi trips.

    The app has three main components:
        1. A datashaded heatmap of taxi trips
        2. A set of widgets controlling the data plotted on the map
        3. A histogram of the number of passengers  per trip

    The histogram is linked to the Holoviews map and only plots the
    histogram of data currently displayed on the map.
    """

    # Map opacity
    alpha = pm.Magnitude(default=0.75, doc="Alpha value for the map opacity")

    # Colormap
    cmap = pm.Selector(cm["fire"], objects={c: cm[c] for c in cmaps})

    # Hour
    hour = pm.Range(default=(0, 24), bounds=(0, 24))

    # Location to plot
    location = pm.Selector(default="dropoff", objects=["dropoff", "pickup"])

    # Selection that gives the current x_range/y_range of the map
    box = hv.streams.RangeXY(x_range=None, y_range=None)

    @pm.depends("location", "hour")
    def get_points_dataset(self, x_range=None, y_range=None):
        """
        Get a Holoviews.Points object holding the taxi data.

        Before returning, filter the points by dropoff hour
        and x/y range.
        """
        # Create the Points object holding all data
        xcol = f"{self.location}_x"
        ycol = f"{self.location}_y"
        points = hv.Points(DATA, kdims=[xcol, ycol], vdims=["dropoff_hour"])

        # Trim by dropoff hour
        if self.hour != (0, 24):
            points = points.select(dropoff_hour=self.hour)

        # Trim by x range of plot
        if x_range is not None:
            points = points.select(**{xcol: x_range})

        # Trim by y range of plot
        if y_range is not None:
            points = points.select(**{ycol: y_range})

        return points

    def pane_heatmap(self, **kwargs):
        """
        Return a datashaded heatmap of the taxi trips.
        """
        # Create a dynamic map using the get_points_dataset() function
        # and link the box selection stream to the map
        points = hv.DynamicMap(self.get_points_dataset, streams=[self.box])

        # aggregate the points by counting them up on a mesh
        agg = rasterize(points, x_sampling=1, y_sampling=1, width=800, height=400)

        # the background tiles
        tiles = gv.tile_sources.CartoDark.apply.opts(alpha=self.param.alpha, **opts)

        # return datashaded heatmap
        heatmap = tiles * shade(agg, cmap=self.param.cmap)

        return heatmap.options(
            default_tools=["save", "pan", "box_zoom", "reset"],
            active_tools=["box_zoom"],
            width=600,
            height=500,
        )

    @pm.depends("hour", "box.x_range", "box.y_range")
    def pane_passenger_hist(self):
        """
        Return an Altair histogram showing the number of passengers
        per trip.

        This chart depends on the box selection's x/y range and will be
        re-drawn when the bounds of the map are updated by the user.
        """

        # get the currently displayed points
        points = self.get_points_dataset(
            x_range=self.box.x_range, y_range=self.box.y_range
        )

        # Get the number of trips for different passenger counts
        N = (
            points.data["passenger_count"]
            .value_counts()
            .rename_axis("Passenger Count")
            .reset_index(name="Total Trips")
        )

        # make the Altair chart
        chart = (
            alt.Chart(N)
            .mark_bar()
            .encode(
                x="Total Trips:Q",
                y="Passenger Count:O",
                tooltip=["Total Trips", "Passenger Count"],
            )
            .properties(width=300, height=400)
        )

        return pn.pane.Vega(chart, width=550)

In [7]:
# initialize our app
app = NYCTaxiApp(name="")

## Layout our Panel object

We can use a combination of the Column() and Row() objects in Panel to create the layout in the main component. We'll also use the [Bootstrap template](https://panel.holoviz.org/reference/templates/Bootstrap.html) to improve the aesthetics of the user interface.

::: {.callout-note}
- The app.param is an automatically generated set of widgets that corresponds to our param parameters
- The charts are specified as the functions of our main application, e.g., app.bar_chart is the function that will return our bar chart.
- Also, we can use the `Spacer()` object to explicitly add more blank space of fixed width/height.
:::

In [8]:
# Set up the dashboard
panel = pn.template.BootstrapTemplate(
    title="Visualizing NYC Taxi Trips with Datashader, Altair, and Panel",
    sidebar_width=250,
)

In [9]:
# Create instructions for the heatmap
instructions = pn.pane.HTML(
    """
<div font-size=28px>
    <b>Note:</b> Zoom or use the box select tool on the heatmap to update the histogram to show results 
    for the taxi trip within the viewing window.
</div>
"""
)

In [10]:
panel.sidebar.append(
    pn.Column(
        pn.pane.Markdown("#### Viewing Options"),
        pn.Param(app.param, expand_button=False, width=200),
    )
)

In [11]:
# Layout the dashboard
panel.main.append(
    pn.Column(pn.Row(app.pane_heatmap, app.pane_passenger_hist, height=550), instructions)
)

### Call servable() and render our Panel object

- The final step is to call the `.servable()` function
- This will render the dashboard directly in the notebook
- It also enables the notebook to be served from `localhost`.

From the command line, we will run:

```
panel serve --show app.ipynb
```

And see the app live at: `http://localhost:5006/app`

In [12]:
panel.servable()

BootstrapTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] BootstrapTemplateActions()
    [browser_info] BrowserInfo()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [6430366048] Column(sizing_mode='stretch_width')
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] Param(NYCTaxiApp, expand_button=False, width=200)
    [6430370176] Column(sizing_mode='stretch_width')
        [0] Row(height=550, sizing_mode='stretch_width')
            [0] ParamMethod(method, _pane=HoloViews, defer_load=False, sizing_mode='stretch_width')
            [1] ParamMethod(method, _pane=Vega, defer_load=False, sizing_mode='stretch_width')
        [1] HTML(str, sizing_mode='stretch_width')